In [1]:
!pip install PyGreSQL
# !pip install nbimporter

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import csv
import pgdb
import pickle
import zipfile
# import nbimporter
import numpy as np
import pandas as pd
import scipy
# import ProviderPredictionScript

from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.multioutput import MultiOutputClassifier

In [3]:
### How many providers do we want to synthesize?

## User defined for each specialty
num_providers = 1000

In [4]:
### Probability of number of claims per provider

### We'll eventually bring inference (of number of claims for each provider) from all customers for each specialty.

### Based on MonteCarlo simulation from the empirical data
num_claims_df = pd.read_csv('num_claims_df.csv')


num_iter = 100000
num_claims_avg_lst = []
for ii in range(0, num_iter):
    num_claims_avg_lst.append(round(num_claims_df.sample(round(0.1*len(num_claims_df))).num_claims.mean()))
dist_mean = np.round(np.mean(num_claims_avg_lst))
dist_std = np.round(np.std(num_claims_avg_lst))
group = [int(round(x)) for x in np.random.normal(loc=dist_mean, scale=3*dist_std,size=num_providers)]
scipy.stats.describe(group)

DescribeResult(nobs=1000, minmax=(52, 234), mean=133.759, variance=854.3592782782782, skewness=0.01724649611566404, kurtosis=-0.11366194637047133)

In [5]:
### Determine probability of different combinations 
### We'll eventually bring inference (of number of claims for each combination) from all customers for each specialty.
all_combinations = pd.read_csv('all_combinations.csv')
sel_combinations = all_combinations[all_combinations.prob_claims>=0.01]

In [6]:
## Unique combinations and corresponding weights
comb_list = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).values
weights = sel_combinations['prob_claims'].tolist()

In [7]:
import random  
final_df_list = []
random_outliers = random.sample(range(0, num_providers), int(round(0.01*num_providers)))
for ii in range(0, num_providers):
    if ii in random_outliers:
        tmp = pd.DataFrame(random.choices(comb_list, random.shuffle(weights), k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    else:
        tmp = pd.DataFrame(random.choices(comb_list, weights, k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    tmp['ren_id'] = 'prov_' + str(ii)
    tmp['prov_claim_id'] = ['claim_' + str(x) for x in range(0, group[ii])]
    final_df_list.append(tmp)
final_df = pd.concat(final_df_list)

In [8]:
final_df[final_df['ren_id'] == 'prov_0']

,proc_values,place_of_service,pat_gender,diag_values,mod_values,ren_id,prov_claim_id
0,90837,30,F,F411,WD,prov_0,claim_0
1,90837,22,F,F411,WD@95,prov_0,claim_1
2,90837,30,None,F411,WD,prov_0,claim_2
3,90837,22,M,F4323,WD@95,prov_0,claim_3
4,90837,30,None,F4312,WD,prov_0,claim_4
...,...,...,...,...,...,...,...
127,90837,22,F,F4322,WD@GT,prov_0,claim_127
128,90837,30,None,F4325,WD,prov_0,claim_128
129,90837,30,None,F411,WD,prov_0,claim_129
130,90837,30,M,F411,WD,prov_0,claim_130


In [9]:
final_df

,proc_values,place_of_service,pat_gender,diag_values,mod_values,ren_id,prov_claim_id
0,90837,30,F,F411,WD,prov_0,claim_0
1,90837,22,F,F411,WD@95,prov_0,claim_1
2,90837,30,None,F411,WD,prov_0,claim_2
3,90837,22,M,F4323,WD@95,prov_0,claim_3
4,90837,30,None,F4312,WD,prov_0,claim_4
...,...,...,...,...,...,...,...
179,90837,30,None,F321,WD,prov_999,claim_179
180,90837@90837@90837,30,None,F411,WD,prov_999,claim_180
181,90834,22,F,F411,WD@GT,prov_999,claim_181
182,90837,22,F,F4323,WD@95,prov_999,claim_182


providers area is alphereta with further away having fewer paitients<br>
<b>work on cost </b><br>
send me a sample members file ecuding phi using age to randomize the date

In [10]:
a = []
for index, row in final_df.iterrows():  
    b = (row['proc_values'].split("@")+ row["mod_values"].split("@"))
    b.sort()
    b = [str(x) for x in b]
    b = "@".join(b)
    a.append(b)
    
final_df["combo"] = a
    
final_df

,proc_values,place_of_service,pat_gender,diag_values,mod_values,ren_id,prov_claim_id,combo
0,90837,30,F,F411,WD,prov_0,claim_0,90837@WD
1,90837,22,F,F411,WD@95,prov_0,claim_1,90837@95@WD
2,90837,30,None,F411,WD,prov_0,claim_2,90837@WD
3,90837,22,M,F4323,WD@95,prov_0,claim_3,90837@95@WD
4,90837,30,None,F4312,WD,prov_0,claim_4,90837@WD
...,...,...,...,...,...,...,...,...
179,90837,30,None,F321,WD,prov_999,claim_179,90837@WD
180,90837@90837@90837,30,None,F411,WD,prov_999,claim_180,90837@90837@90837@WD
181,90834,22,F,F411,WD@GT,prov_999,claim_181,90834@GT@WD
182,90837,22,F,F4323,WD@95,prov_999,claim_182,90837@95@WD


In [11]:
cost = pd.read_csv("amt_df.csv")

cost

,proc_values,diag_values,mod_values,place_of_service,pat_gender,amount_charged,amount_paid
0,90834@90834,F4322,WD,30,None,100.0,94.95
1,90837,F4325,WD,30,None,110.0,67.28
2,90837,F4310,WD,30,None,110.0,89.70
3,90837@90837@90791@90837@90837@90837,F329,WD,30,None,100.0,89.70
4,90837@90837@90837@90837@90837@90837@90837@9083...,F341,WD,30,None,130.0,89.70
...,...,...,...,...,...,...,...
1370057,90837@90837@90837@90837@90837@90837@90837@9083...,F341,WD,30,M,150.0,0.00
1370058,90837@90837,F4312,WD@GT,22,F,120.0,109.00
1370059,90837,F419,WD,30,M,175.0,0.00
1370060,90834@90834@90834@90834@90834@90834@90834@9083...,F411,WD@95:WD@95:WD@95,22,F,300.0,0.00


In [12]:
a = []
for index, row in cost.iterrows():  
    b = (row['proc_values'].split("@")+ row["mod_values"].split("@"))
    b.sort()
    b = [str(x) for x in b]
    b = "@".join(b)
    a.append(b)
    
cost["combo"] = a
cost = cost.drop(columns=['diag_values', 'place_of_service',"pat_gender", "mod_values", "proc_values"])
cost.head()


,amount_charged,amount_paid,combo
0,100.0,94.95,90834@90834@WD
1,110.0,67.28,90837@WD
2,110.0,89.70,90837@WD
3,100.0,89.70,90791@90837@90837@90837@90837@90837@WD
4,130.0,89.70,90837@90837@90837@90837@90837@90837@90837@9083...


In [13]:
c = cost.groupby(['combo'])['amount_charged'].sum()
c = c.reset_index()
e = cost.groupby(['combo'],as_index = False).size()
c["count"] = e["size"]
c["avg_charge"] = c["amount_charged"]/ c["count"]

####
ds = cost[cost.amount_paid !=0]
d = ds.groupby(['combo'])['amount_paid'].sum()
d = d.reset_index()
e = ds.groupby(['combo'], as_index= False).size()
d["count"] = e["size"]
d = d.merge(c[["combo", "avg_charge"]])
d["avg_paid_pct"] = (d["amount_paid"]/ d["count"])/d["avg_charge"]


####
f = cost.groupby(['combo'])['amount_paid'].sum()
f = f.reset_index()
e = cost.groupby(['combo'])['amount_paid'].size()
e = e.reset_index()
f["count"] = e["amount_paid"]
f = f.merge(c[["combo", "avg_charge"]])
f["avg_paid_pct"] = (f["amount_paid"]/ f["count"])/f["avg_charge"]
f = f[f.avg_paid_pct == 0.000000]

d = d.append(f)
d

,combo,amount_paid,count,avg_charge,avg_paid_pct
0,00145@00145@00145@00145@QS@QX@WJ,122.11,1,1100.000000,0.111009
1,1000F@1000F@1000F@1160F@1160F@1160F@3074F@3074...,89.70,1,144.000000,0.622917
2,1036F@90791@G8417@G8428@G9903@WD,85.19,1,256.000000,0.332773
3,1036F@90791@None:WD,402.60,2,376.030000,0.535330
4,1036F@90832@95:None:None:None:None@G8417@G8428...,36.59,1,85.000000,0.430471
...,...,...,...,...,...
13539,S9484@S9484@WD,0.00,2,72.050000,0.000000
13540,S9484@S9484@WD:WD,0.00,1,73.800000,0.000000
13541,S9484@WD,0.00,12,110.116667,0.000000
13542,T1019@T1019@T1019@T1019@T1019@U5:U5:U5:U5:U5,0.00,1,263.200000,0.000000


In [14]:
scb = 50#may want to change scale so that it is more in line with reality
sca = .05

cost_dict = c.set_index('combo').to_dict()
final_df['cost'] = final_df['combo'].apply(lambda x: cost_dict["avg_charge"][x])
final_df['real charge'] = np.random.normal(final_df['cost'], scale = scb).astype(int)
final_df['real charge'] = final_df['real charge'].abs()
final_df = final_df.drop(columns=['cost'])


paid_dict = d.set_index('combo').to_dict()
final_df['paid'] = final_df['combo'].apply(lambda x: paid_dict["avg_paid_pct"][x])
final_df['real paid'] = np.random.normal(final_df['paid'], scale = sca)*final_df["real charge"].astype(int)
final_df['real paid'] = final_df['real paid'].abs()

final_df = final_df.drop(columns=['paid'])
final_df["real paid"] = final_df["real paid"].astype(int)

final_df.to_csv("final.csv")

final_df

,proc_values,place_of_service,pat_gender,diag_values,mod_values,ren_id,prov_claim_id,combo,real charge,real paid
0,90837,30,F,F411,WD,prov_0,claim_0,90837@WD,145,105
1,90837,22,F,F411,WD@95,prov_0,claim_1,90837@95@WD,111,79
2,90837,30,None,F411,WD,prov_0,claim_2,90837@WD,87,58
3,90837,22,M,F4323,WD@95,prov_0,claim_3,90837@95@WD,141,90
4,90837,30,None,F4312,WD,prov_0,claim_4,90837@WD,194,148
...,...,...,...,...,...,...,...,...,...,...
179,90837,30,None,F321,WD,prov_999,claim_179,90837@WD,190,124
180,90837@90837@90837,30,None,F411,WD,prov_999,claim_180,90837@90837@90837@WD,175,122
181,90834,22,F,F411,WD@GT,prov_999,claim_181,90834@GT@WD,101,50
182,90837,22,F,F4323,WD@95,prov_999,claim_182,90837@95@WD,197,139
